In [1]:
import pandas as pd
import numpy as np
import requests
import json
from pandas import json_normalize
from requests.auth import HTTPBasicAuth

from dotenv.main import load_dotenv
import os



pd.options.display.max_seq_items = 20000
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)

In [24]:
#Load the Api Key

from dotenv.main import load_dotenv
import os
load_dotenv()
API_KEY=os.environ["api_key"]

### 0.1 Notes and Thoughts

Hard Filters that are needed for the Dataset are:
- B2C
- Europe 
- (Later industry, first iteration without industry specificat)

All Available fields:
id
name
type
deleted
path
tagline
about
url,website_url,twitter_url,facebook_url,linkedin_url,google_url,crunchbase_url,angellist_url
playmarket_app_id,appstore_app_id
images
employees,employees_latest
industries,sub_industries,corporate_industries,service_industries
technologies
income_streams
growth_stage
traffic_summary
hq_locations
client_focus
revenues
tags
ownerships
delivery_method
launch_year,launch_month
has_promising_founder,has_strong_founder,has_super_founder
total_funding,total_funding_source,last_funding,last_funding_source
company_status,last_updated,last_updated_utc,created_utc
facebook_likes_chart,twitter_tweets_chart,twitter_followers_chart,twitter_favorites_chart
employees_chart
similarweb_3_months_growth_unique,similarweb_3_months_growth_percentile,similarweb_3_months_growth_relative,similarweb_3_months_growth_delta,similarweb_6_months_growth_unique,similarweb_6_months_growth_percentile,similarweb_6_months_growth_relative,similarweb_6_months_growth_delta,similarweb_12_months_growth_unique,similarweb_12_months_growth_percentile,similarweb_12_months_growth_relative,similarweb_12_months_growth_delta

app_3_months_growth_unique,app_3_months_growth_percentile,app_3_months_growth_relative,app_6_months_growth_unique,app_6_months_growth_percentile,app_6_months_growth_relative,app_12_months_growth_unique,app_12_months_growth_percentile,app_12_months_growth_relative,employee_3_months_growth_unique

employee_3_months_growth_percentile,employee_3_months_growth_relative,employee_3_months_growth_delta,employee_6_months_growth_unique,employee_6_months_growth_percentile,employee_6_months_growth_relative,employee_6_months_growth_delta,employee_12_months_growth_unique,employee_12_months_growth_percentile,employee_12_months_growth_relative,employee_12_months_growth_delta
kpi_summary
team
investors
fundings
traffic
similarweb_chart
job_openings
exits
trading_multiple
app_downloads_ios_chart,app_downloads_android_chart,app_downloads_ios_incremental_chart,app_downloads_android_incremental_chart
tech_stack_predictleads
sustainable_development_goals
core_side_value
data_type
pic_number
patents_count


### 0.2 Getting Available Filters for the Post Requests

In [27]:

api_url="https://api.dealroom.co/api/v1/companies/filters"
auth=HTTPBasicAuth(API_KEY, '')

headers = {'Content-Type': 'application/json'}
data={"fields":"hq_locations",
              "limit": 1,
              "offset": 0
             }
response = requests.post(api_url,data=json.dumps(data),auth=auth,headers=headers)
data=response.json()



# 1.1 Building the Core DF

1. Variables for the DF:

1.1: Independent Variables
- Team
- patents (patents_count)
- Round (fundings)
- Investors
- employees (latest, chart)
- Num. Investors (investors)
- (Industry Category) (industries)
- Description Length (about)
- Has Domain (website_url)
- Has Facebook (facebook_url)
- Has Twitter (twitter_url)
- Has LinkedIn (linkedin_url)
- Geographical Region (country, city/-code) (hq_locations)
- Total Funding (total_funding) (in mio)

1.2 Dependent Variables:
- Acquisition (company_status)
- Failure (company_status)
- Public (ownerships)

### Importing the Json file through according API Call:

In [36]:
fields="id,name,about,tagline,investors,industries,website_url,facebook_url,twitter_url,linkedin_url,hq_locations,total_funding,company_status,ownerships,team,fundings,patents_count,employees_latest,employees_chart"
api_url="https://api.dealroom.co/api/v1/companies/bulk"
auth=HTTPBasicAuth(API_KEY, '')
headers = {'Content-Type': 'application/json'}

json_list=[]
next_page_id = ''
k=0

while True:
    data = {
        'form_data': {'must':{'hq_locations': ['Europe'], 'client_focus':['business']}},
        'fields': fields,
        'next_page_id': next_page_id,
        'limit': 100,
        }

    headers = {'Content-Type': 'application/json'}

    response = requests.post(api_url,data=json.dumps(data),auth=auth,headers=headers)

    res=json.loads(response.text)
    json_list.append(res)
    k+=1
    k
    next_page_id = res['next_page_id']

    if next_page_id == None:
        break

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

2167

In [38]:
with open("first_json.json", "w") as fp:
        json.dump(json_list, fp)

In [25]:
f=open("/Users/klaus/Documents/Jarvis/Second Try/Data/first_json.json")
json_list=json.load(f)

In [27]:
pd.json_normalize(json_list[0]["items"])


,id,name,about,tagline,industries,website_url,facebook_url,twitter_url,linkedin_url,hq_locations,total_funding,company_status,ownerships,patents_count,employees_latest,employees_chart,investors.items,investors.total,team.items,team.total,fundings.items,fundings.total
0,2548773,Linton Recruitment Solutions,We opened our doors in 2015 with the desire to...,Linton Recruitment Solutions – Facilities Mana...,"[{'id': 100108, 'name': 'jobs recruitment'}]",http://lintonrs.com,None,https://twitter.com/lintonrs,https://www.linkedin.com/company/linton-recrui...,"[{'id': 2357181, 'address': 'London, England, ...",0,operational,[],0,7.0,"[{'date': '2021-03-30', 'value': 8}, {'date': ...",[],0,[],0,[],0
1,2554922,Lingwella Translation Agency,"Established in 2009, Lingwella Translation Age...",None,"[{'id': 100108, 'name': 'jobs recruitment'}]",http://lingwella.com,None,None,https://www.linkedin.com/company/lingwella/,"[{'id': 2297595, 'address': 'Dimitrovgrad, Vol...",0,operational,[],0,1.0,"[{'date': '2020-01-21', 'value': 1}, {'date': ...",[],0,[],0,[],0
2,3625129,LOOK FACADE,LOOK FACADE is a management consulting company...,Materials experts to fulfill your design,[],http://lookfacade.com,None,None,https://www.linkedin.com/company/look-facade,"[{'id': 3108373, 'address': 'Paris, Ile-de-Fra...",0,operational,[],0,2.0,"[{'date': '2022-02-08', 'value': 2}, {'date': ...",[],0,[],0,[],0
3,1968146,Liveforce,On-Demand Workforce Software Made Simple — A n...,All-in-one Event and Promotional Staffing Soft...,"[{'id': 100108, 'name': 'jobs recruitment'}]",http://liveforce.co,None,https://twitter.com/liveforceapp,https://www.linkedin.com/company/liveforce,"[{'id': 2282791, 'address': 'Surrey Lane, Lond...",0,operational,[bootstrapped],0,18.0,"[{'date': '2020-12-18', 'value': 10}, {'date':...",[],0,"[{'id': 3794983, 'name': 'Mark Walls', 'path':...",2,[],0
4,2122413,Gorilla,"Discover, visualize and use social content and...",Livereach - Turn Social Data into Impactful Ex...,"[{'id': 100147, 'name': 'enterprise software'}]",http://livereach.com,None,https://twitter.com/livereach,https://www.linkedin.com/company/livereach/,"[{'id': 2062006, 'address': 'Geisbergweg 8, 48...",0,operational,[bootstrapped],0,7.0,"[{'date': '2021-03-08', 'value': 16}, {'date':...",[],0,"[{'id': 4303626, 'name': 'Maarten Janello', 'p...",1,[],0
5,2128966,Lilas,None,LILAS GmbH | A research and development compan...,"[{'id': 100111, 'name': 'transportation'}]",http://lilas.de,None,None,https://www.linkedin.com/company/lilas-gmbh/,"[{'id': 2072608, 'address': 'Carlo-Schmid-Alle...",0,operational,[],21,2.0,"[{'date': '2021-07-05', 'value': 2}, {'date': ...",[],0,[],0,[],0
6,3595455,Loadero,Loadero is a cloud-hosted service for testing ...,Run end-to-end performance and load tests for ...,[],http://loadero.com,None,None,https://www.linkedin.com/company/loadero,"[{'id': 3087446, 'address': 'Riga, Vidzeme, La...",0,operational,[],0,8.0,"[{'date': '2022-02-07', 'value': 7}, {'date': ...",[],0,[],0,[],0
7,3340355,Locum Booking,"Locum Booking Platforms for GPs, Pharmacists, ...",None,"[{'id': 1254, 'name': 'health'}]",http://locumbooking.com,None,https://twitter.com/locumbooking,https://www.linkedin.com/company/locumbooking,"[{'id': 3434099, 'address': 'Hythe Street, Tem...",0,operational,[],0,NaN,[],[],0,[],0,[],0
8,2536993,Logic architecture,Logic architecture is RIBA Chartered architect...,Logic Architecture – North East Based Chartere...,"[{'id': 1270, 'name': 'real estate'}]",http://logicarchitecture.com,None,https://twitter.com/logic_arch,https://www.linkedin.com/company/logic-archite...,"[{'id': 3407620, 'address': 'Bridge Street Wes...",0,operational,[],0,10.0,"[{'date': '2021-03-30', 'value': 13}, {'date':...",[],0,[],0,[],0
9,2074059,Lobusch Consulting,We are a professional company providing consul...,Legal and Accounting Services Company in Ukrai...,[],http://lobusch-consulting.com,None,None,https://www.linkedin.com/company/lobusch-consu...,"[{'id': 2585447, 'address': 'Ц

### 1.2 Converting Json to Pandas DF

In [21]:
df_list=[]
for i in range(len(json_list)):
    df_1=pd.json_normalize(json_list[i]["items"],sep="_").drop(columns=["hq_locations","industries"])
    df_1[["country_name","city_name"]]=pd.json_normalize(json_list[0]["items"],"hq_locations")[["country.name","city.name"]]
    df_1["industry_name"]=pd.json_normalize(json_list[0]["items"],"industries")["name"]
    df_list.append(df_1)

KeyError: 0

In [46]:
df=pd.concat(df_list)

In [47]:
df.dtypes

id                    int64
name                 object
about                object
tagline              object
website_url          object
facebook_url         object
twitter_url          object
linkedin_url         object
total_funding       float64
company_status       object
ownerships           object
patents_count         int64
employees_latest    float64
employees_chart      object
investors_items      object
investors_total       int64
team_items           object
team_total            int64
fundings_items       object
fundings_total        int64
country_name         object
city_name            object
industry_name        object
dtype: object

## 1.3 DataCleaning

In [52]:
x=df.isna().sum()
y=df.isna().sum()/len(df)*100
missing_values=pd.concat([x, y], axis=1)
missing_values.columns=["Missing Values","In percent"]
print(missing_values)


NameError: name 'df' is not defined

In [50]:
df.dropna(subset=["website_url"],inplace=True) #companies without website will be deleted to avoid ghost companies
df.dropna(subset=["company_status"],inplace=True) #companies without status (only 85 so can be dismissed)

### Number of Empty lists as entries

In [51]:
for i,row in missing_values.iterrows():
    try:
        print(i,df[i].apply(lambda x:1 if len(x) == 0 else 0).sum()/len(df)*100,"%")
    except:
        None



name 0.0 %
website_url 0.0 %
company_status 0.0 %
ownerships 56.60068145624236 %
employees_chart 27.224917595644605 %
investors_items 75.60831080330358 %
team_items 61.55049812969891 %
fundings_items 75.89302248064887 %
country_name 0.0 %


In [52]:
df_1=df[df["ownerships"].apply(lambda x:False if len(x) == 0 else True)]

In [53]:
df_2=df[df["fundings_items"].apply(lambda x:False if len(x) == 0 else True)]

In [54]:
for i,row in missing_values.iterrows():
    try:
        print(i,df_1[i].apply(lambda x:1 if len(x) == 0 else 0).sum()/len(df)*100,"%")
    except:
        None


for i,row in missing_values.iterrows():
    try:
        print(i,df_2[i].apply(lambda x:1 if len(x) == 0 else 0).sum()/len(df)*100,"%")
    except:
        None       

print(df_1.shape)
print(df_2.shape)

name 0.0 %
website_url 0.0 %
company_status 0.0 %
ownerships 0.0 %
employees_chart 7.195566830858116 %
investors_items 27.0962186585682 %
team_items 9.497333432095107 %
fundings_items 23.77226769378912 %
country_name 0.0 %
name 0.0 %
website_url 0.0 %
company_status 0.0 %
ownerships 4.479926669382615 %
employees_chart 3.1744194659457055 %
investors_items 3.9225399059294097 %
team_items 7.764527239731862 %
fundings_items 0.0 %
country_name 0.0 %
(93746, 23)
(52073, 23)


When removing only companies with missing fundings we receive a set of 52.073 companies and 7.7% missing teams (set.1)
When removing only ownerships we receive a set of 93.746 with 9.5% missing teams (set.2)
As the fundings are only relevant when trying to predict probability of follow on fundings we will continue using set.2 and use type of company status as dependent variable

### Further clean Dataset by removing all Empty lists in the team column

In [55]:
df_1=df_1[df_1["team_items"].apply(lambda x:False if len(x) == 0 else True)]
df_1.shape

(73231, 23)

In [56]:
for i,row in missing_values.iterrows():
    try:
        print(i,df_1[i].apply(lambda x:1 if len(x) == 0 else 0).sum()/len(df)*100,"%")
    except:
        None

name 0.0 %
website_url 0.0 %
company_status 0.0 %
ownerships 0.0 %
employees_chart 3.318395614977223 %
investors_items 21.779748898188956 %
team_items 0.0 %
fundings_items 19.935372763971703 %
country_name 0.0 %


### We observe that many of the empty lists for fundings and investors are for bootstrapped startups. It makes sense to distinguish between those where data is missing and those where it simply makes no sense to have data

In [89]:
df_invest=df_1.loc[df_1["investors_items"].apply(lambda x:True if len(x) == 0 else False)]
df_boots=df_invest.loc[df_invest["ownerships"].apply(lambda x:True if ((x[0]=="bootstrapped" and len(x)<=1) or (x==["not known","bootstrapped"]) or (x[0]=="not known")) else False)]

df_boots["investors_items"].values[:]="nan"

df_2=pd.concat([df_1,df_boots])

df_2.drop_duplicates(subset=["id"],keep="last", inplace=True)

In [92]:
df_fundings=df_2.loc[df_2["fundings_items"].apply(lambda x:True if len(x) == 0 else False)]
df_fundinv=df_fundings.loc[df_fundings["investors_items"].apply(lambda x:True if x == "nan" else False)]
df_shit=df_fundinv.loc[df_fundinv["ownerships"].apply(lambda x:True if ((x[0]=="bootstrapped" and len(x)<=1) or (x==["not known","bootstrapped"]) or (x[0]=="not known")) else False)]

df_shit["fundings_items"].values[:]="nan"

df_3=pd.concat([df_2,df_shit])

df_3.drop_duplicates(subset=["id"],keep="last", inplace=True)

In [69]:
for i,row in missing_values.iterrows():
    try:
        print(i,df_3[i].apply(lambda x:1 if (len(x) == 0 or x==None) else 0).sum()/len(df_3)*100,"%")
    except:
        None

for i,row in missing_values.iterrows():
    try:
        print(i,df_3[i].apply(lambda x:1 if len(x) == 0 else 0).sum())
    except:
        None

name 0.0 %
website_url 0.0 %
company_status 0.0 %
ownerships 0.0 %
employees_chart 9.78820444893556 %
investors_items 5.236853245210362 %
team_items 0.0 %
fundings_items 1.5922218732503994 %
country_name 0.0 %
team_universities 0.0 %
name 0
website_url 0
company_status 0
ownerships 0
employees_chart 7168
investors_items 3835
team_items 0
fundings_items 1166
country_name 0
team_universities 0


Data Seems quite clean to me. No more Data Cleaning Needed 

### 1.3 Unravel nested Data

For the following we want to extract the nested data and retreive the information we need
First we extract investors types

In [194]:
first=pd.json_normalize(df_3.iloc[2]["team_items"])["universities.items"]
mppd.json_normalize(a.iloc[3]) #university.name and age

,id,degree,majors,year_start,year_end,university.id,university.name,university.path,university.type,university.url,university.images.32x32,university.images.74x74,university.images.100x100,degree.id,degree.name,degree.years
0,441645,NaN,[],2015,2016.0,936807,CIM | The Chartered Institute of Marketing,cim_the_chartered_institute_of_marketing,university,https://app.dealroom.co/universities/cim_the_c...,https://s3-eu-west-1.amazonaws.com/dealroom-im...,https://s3-eu-west-1.amazonaws.com/dealroom-im...,https://s3-eu-west-1.amazonaws.com/dealroom-im...,NaN,NaN,NaN
1,441656,NaN,[],2012,NaN,4163126,SRH Berlin School of Design and Communication,srh_berlin_school_of_design_and_communication,university,https://app.dealroom.co/universities/srh_berli...,https://s3-eu-west-1.amazonaws.com/dealroom-im...,https://s3-eu-west-1.amazonaws.com/dealroom-im...,https://s3-eu-west-1.amazonaws.com/dealroom-im...,4.0,Bachelor,3.0


In [72]:
def level1_extractor_list(x,field):
    try:
       return list(pd.json_normalize(x)[field])
    except:
        return None

def level1_extractor_max(x,field):
    try:
        a=max(list(pd.json_normalize(x)[field]))
    except:
        a=None
    return a

def level2_extractor_list(x,field1,field2):
    try:
        first=pd.json_normalize(x)[field1]
        mp=map(lambda p: level1_extractor_list(p, field2), first)
        a=[x for x in list(mp) if x!=None]
    except:
        a=None
    return a

def level2_extractor_max(x,field1,field2):
    try:
        first=pd.json_normalize(x)[field1]
        mp=map(lambda p: level1_extractor_list(p, field2), first)
        a=max([x for x in list(mp) if x!=None])
    except:
        a=None
    return a

def level2_extractor_min(x,field1,field2):
    try:
        first=pd.json_normalize(x)[field1]
        mp=map(lambda p: level1_extractor_list(p, field2), first)
        a=min([x for x in list(mp) if x!=None])
    except:
        a=None
    return a

def level2_extractor_avg(x,field1,field2):
    try:
        first=pd.json_normalize(x)[field1]
        mp=map(lambda p: level1_extractor_list(p, field2), first)
        a=np.mean([x for x in list(mp) if x!=None])
    except:
        a=None
    return a

Export Feature of Universities at which team members studies

In [73]:
a=map(lambda p: level2_extractor_list(p, "universities.items","university.name"), df_3["team_items"])
b=list(a)

df_3["team_universities"]=b

Export Features of Investor Types

In [56]:
a=map(lambda p: level1_extractor_list(p, "type"), df_3["investors_items"])
b=list(a)
df_3["investors_types"]=b

In [249]:
#Save Dataframe
df_3.to_pickle("df_25.08")

In [95]:
df_3=pd.read_pickle("/Users/klaus/Documents/Jarvis/Second Try/Data/df_25.08")

Export Features of Valuation

In [45]:
a=map(lambda p: level1_extractor_max(p, "valuation_generated_list"), df_3["fundings_items"])
b=list(a)
df_3["list_valuation"]=b
df_3["last_valuation"]=list(map(lambda p: p[-1],df_3["list_valuation"]))

Get Number of Female and Number of Male Founder

In [91]:
a=map(lambda p: level1_extractor_list(p, "gender"), df_3["team_items"])
b=list(a)
df_3["team_gender"]=b
df_3["team_male"]=list(map(lambda p: p.count("male"),df_3["team_gender"]))
df_3["team_female"]=list(map(lambda p: p.count("female"),df_3["team_gender"]))
df_3.drop(columns="team_gender",inplace=True)

Turning url's to the binary (0-has no url, 1-has url)

In [98]:
df_3["website_url"]=df_3["website_url"].apply(lambda x:0 if x==None else 1)
df_3["linkedin_url"]=df_3["linkedin_url"].apply(lambda x:0 if x==None else 1)
df_3["facebook_url"]=df_3["facebook_url"].apply(lambda x:0 if x==None else 1)
df_3["twitter_url"]=df_3["twitter_url"].apply(lambda x:0 if x==None else 1)

In [100]:
df_3.head(n=20)

,id,name,about,tagline,website_url,facebook_url,twitter_url,linkedin_url,total_funding,company_status,ownerships,patents_count,employees_latest,employees_chart,investors_items,investors_total,team_items,team_total,fundings_items,fundings_total,country_name,city_name,industry_name,investors_types,team_universities
82,2526214,Phoenix Eleven,We empower entrepreneurs giving them the tools...,Phoenix Eleven Capital - The Human Touch,1,0,0,1,0.00,operational,[bootstrapped],0,2.0,"[{'date': '2020-12-01', 'value': 3}, {'date': ...","[{'id': 3214034, 'name': 'Flori Ventures', 'pa...",1,"[{'id': 2969484, 'name': 'Nick Allison', 'path...",2,[],0,Germany,Ravensburg,semiconductors,[fund],[]
84,2441129,Pickar,We believe that finding yourself in a career y...,Developed a psychometric analysis alongside ex...,1,0,0,1,0.00,operational,[Venture Capital],0,1.0,"[{'date': '2021-05-11', 'value': 2}, {'date': ...","[{'id': 935917, 'name': 'Georgia Institute of ...",1,"[{'id': 3831057, 'name': 'Julian Willner', 'pa...",2,"[{'id': 427762, 'year': 2019, 'month': 6, 'amo...",2,Switzerland,Eysins,media,[university],[]
56,1496036,Pro,Pro is a specialist outsourcing and consulting...,Specialist outsourcing and consulting company ...,1,0,1,1,0.00,acquired,[Subsidiary],0,285.0,"[{'date': '2019-03-01', 'value': 243}, {'date'...","[{'id': 1501621, 'name': 'Tawa', 'path': 'tawa...",1,"[{'id': 270510, 'name': 'Richard Lawson', 'pat...",4,"[{'id': 211612, 'year': 2009, 'month': 11, 'am...",1,United Kingdom,NaN,enterprise software,[corporate],"[[CIM | The Chartered Institute of Marketing, ..."
89,155526,QRcao,None,Co-founder and product engineer @ Appvetica | ...,1,1,1,1,0.01,closed,[Venture Capital],0,NaN,[],[],0,"[{'id': 1044949, 'name': 'Peter Tuszynski', 'p...",1,"[{'id': 200613, 'year': 2011, 'month': 8, 'amo...",1,United Kingdom,London,robotics,nan,[]
97,149265,Tech'4'Team,Arenametrix is a SaaS B2B start-up created in ...,None,1,0,1,1,0.60,operational,"[Angel, Venture Capital, accelerator]",0,26.0,"[{'date': '2018-07-16', 'value': 19}, {'date':...","[{'id': 885468, 'name': 'Paris&Co', 'path': 'i...",2,"[{'id': 349656, 'name': 'Vitoz Kévin', 'path':...",6,"[{'id': 222831, 'year': 2016, 'month': 4, 'amo...",2,United Kingdom,NaN,NaN,"[fund, investor]",[]
11,118206,Newswired,None,Digital publishing platform that enables publi...,1,1,1,1,0.02,closed,[Venture Capital],0,NaN,[],[],0,"[{'id': 1135128, 'name': 'Andrey Aleshkov', 'p...",2,"[{'id': 202314, 'year': 2013, 'month': 12, 'am...",1,United Kingdom,Great and Little Hampden,transportation,nan,[]
97,97110,RentBid,Mechanics Mechanics Pick Your yesMechanics Vie...,None,1,0,1,1,0.00,closed,[Venture Capital],0,NaN,[],"[{'id': 908352, 'name': 'Rev1 Ventures', 'path...",1,"[{'id': 255915, 'name': 'Alexander Willcox', '...",2,"[{'id': 288441, 'year': 2015, 'month': 5, 'amo...",1,United Kingdom,NaN,NaN,[fund],[]
99,98045,Overture Services,Home | Yahoo Advertising Home Mail Search New...,Online advertising services via its network of...,1,1,1,1,0.00,acquired,"[Subsidiary, Angel, Venture Capital]",0,NaN,[],"[{'id': 20810, 'name': 'Upfront Ventures', 'pa...",4,"[{'id': 258014, 'name': 'Laurence Aderemi', 'p...",28,"[{'id': 226763, 'year': 2003, 'month': 7, 'amo...",3,United Kingdom,West Suffolk,NaN,"[fund, corporate, investor, fund]",[]
0,98343,InvisibleHand,InvisibleHand is one of the world's most popul...,None,1,1,1,1,0.00,acquired,"[Subsidiary, Venture Capital]",0,1.0,"[{'date': '2021-09-17', 'value': 1}, {'date': ...","[{'id': 20264, 'name': 'skimlinks', 'path': 's...",1,"[{'id': 51775, 'name': 'Evgeny Shadchnev', 'pa...",3,"[{'id': 359332, 'year': 2016, 'month': 1, 'amo...",2,United Kingdom,London,jobs recruitment,[corporate],"[[Voronezh State University, Voronezh State Un..."
1,99046,Coupoplaces,None,A platform where the customers of daily deal c...,1,0,1,1,0.01,closed,[Venture Capital],0,NaN,[],[],0,"[{'id': 1104205, 'name': 'Alex Petrov', 'path'...",3,"[{'id': 15405, 'year': 2011, 'month': 6, 'amou...",1,Russia,Di

In [99]:
pd.json_normalize(df_3.iloc[0]["team_items"])

,id,name,path,type,url,backgrounds,gender,is_serial_founder,linkedin_url,titles,past,is_founder,year_start,year_end,is_strong_founder,is_super_founder,is_promising_founder,images.32x32,images.74x74,images.100x100,universities.items,universities.total
0,2969484,Nick Allison,nick_allison,user,https://app.dealroom.co/people/nick_allison,[],male,False,https://www.linkedin.com/in/pecapital/,"[{'id': 5, 'name': 'CEO'}, {'id': 15, 'name': ...",False,True,2019.0,None,False,False,False,https://s3.eu-west-2.amazonaws.com/dealroom-st...,https://s3.eu-west-2.amazonaws.com/dealroom-st...,https://s3.eu-west-2.amazonaws.com/dealroom-st...,[],0
1,2973123,Dylan Lee,dylan_lee_1,investor,https://app.dealroom.co/investors/dylan_lee_1,[],male,False,https://www.linkedin.com/in/dylan-lee-335a78101/,"[{'id': 462, 'name': 'Business Development Dir...",False,False,NaN,None,False,False,False,https://s3.eu-west-2.amazonaws.com/dealroom-st...,https://s3.eu-west-2.amazonaws.com/dealroom-st...,https://s3.eu-west-2.amazonaws.com/dealroom-st...,[],0


Generate List for LinkedIn Scraper to get Universities of Founders

In [127]:
df_missing_univ=df_3[df_3["team_universities"].apply(lambda x:True if len(x)==0 else False)].copy()

In [128]:
df_missing_univ.shape

(27035, 25)

In [130]:
a=map(lambda p: level1_extractor_list(p, "linkedin_url"), df_missing_univ["team_items"])
b=list(a)
df_missing_univ["linkedin_url_founder"]=b
df_missing_univ["linkedin_url_founder"]=list(map(lambda p: p[0],df_missing_univ["linkedin_url_founder"]))

In [131]:
df_missing_univ.head(n=20)

,id,name,about,tagline,website_url,facebook_url,twitter_url,linkedin_url,total_funding,company_status,ownerships,patents_count,employees_latest,employees_chart,investors_items,investors_total,team_items,team_total,fundings_items,fundings_total,country_name,city_name,industry_name,investors_types,team_universities,linkedin_url_founder
82,2526214,Phoenix Eleven,We empower entrepreneurs giving them the tools...,Phoenix Eleven Capital - The Human Touch,1,0,0,1,0.00,operational,[bootstrapped],0,2.0,"[{'date': '2020-12-01', 'value': 3}, {'date': ...","[{'id': 3214034, 'name': 'Flori Ventures', 'pa...",1,"[{'id': 2969484, 'name': 'Nick Allison', 'path...",2,[],0,Germany,Ravensburg,semiconductors,[fund],[],https://www.linkedin.com/in/pecapital/
84,2441129,Pickar,We believe that finding yourself in a career y...,Developed a psychometric analysis alongside ex...,1,0,0,1,0.00,operational,[Venture Capital],0,1.0,"[{'date': '2021-05-11', 'value': 2}, {'date': ...","[{'id': 935917, 'name': 'Georgia Institute of ...",1,"[{'id': 3831057, 'name': 'Julian Willner', 'pa...",2,"[{'id': 427762, 'year': 2019, 'month': 6, 'amo...",2,Switzerland,Eysins,media,[university],[],https://www.linkedin.com/in/julian-willner/
89,155526,QRcao,None,Co-founder and product engineer @ Appvetica | ...,1,1,1,1,0.01,closed,[Venture Capital],0,NaN,[],[],0,"[{'id': 1044949, 'name': 'Peter Tuszynski', 'p...",1,"[{'id': 200613, 'year': 2011, 'month': 8, 'amo...",1,United Kingdom,London,robotics,nan,[],https://www.linkedin.com/in/petertuszynski/
97,149265,Tech'4'Team,Arenametrix is a SaaS B2B start-up created in ...,None,1,0,1,1,0.60,operational,"[Angel, Venture Capital, accelerator]",0,26.0,"[{'date': '2018-07-16', 'value': 19}, {'date':...","[{'id': 885468, 'name': 'Paris&Co', 'path': 'i...",2,"[{'id': 349656, 'name': 'Vitoz Kévin', 'path':...",6,"[{'id': 222831, 'year': 2016, 'month': 4, 'amo...",2,United Kingdom,NaN,NaN,"[fund, investor]",[],None
11,118206,Newswired,None,Digital publishing platform that enables publi...,1,1,1,1,0.02,closed,[Venture Capital],0,NaN,[],[],0,"[{'id': 1135128, 'name': 'Andrey Aleshkov', 'p...",2,"[{'id': 202314, 'year': 2013, 'month': 12, 'am...",1,United Kingdom,Great and Little Hampden,transportation,nan,[],https://www.linkedin.com/in/andreyaleshkov/
97,97110,RentBid,Mechanics Mechanics Pick Your yesMechanics Vie...,None,1,0,1,1,0.00,closed,[Venture Capital],0,NaN,[],"[{'id': 908352, 'name': 'Rev1 Ventures', 'path...",1,"[{'id': 255915, 'name': 'Alexander Willcox', '...",2,"[{'id': 288441, 'year': 2015, 'month': 5, 'amo...",1,United Kingdom,NaN,NaN,[fund],[],None
99,98045,Overture Services,Home | Yahoo Advertising Home Mail Search New...,Online advertising services via its network of...,1,1,1,1,0.00,acquired,"[Subsidiary, Angel, Venture Capital]",0,NaN,[],"[{'id': 20810, 'name': 'Upfront Ventures', 'pa...",4,"[{'id': 258014, 'name': 'Laurence Aderemi', 'p...",28,"[{'id': 226763, 'year': 2003, 'month': 7, 'amo...",3,United Kingdom,West Suffolk,NaN,"[fund, corporate, investor, fund]",[],None
1,99046,Coupoplaces,None,A platform where the customers of daily deal c...,1,0,1,1,0.01,closed,[Venture Capital],0,NaN,[],[],0,"[{'id': 1104205, 'name': 'Alex Petrov', 'path'...",3,"[{'id': 15405, 'year': 2011, 'month': 6, 'amou...",1,Russia,Dimitrovgrad,jobs recruitment,nan,[],None
4,99372,Socialance,Socialance is a professional network that conn...,Professional network for outsourcing,1,1,1,1,0.04,operational,[Venture Capital],0,NaN,[],"[{'id': 3654, 'name': 'Start-Up Chile', 'path'...",1,"[{'id': 1148514, 'name': 'Guillermo Vigil', 'p...",2,"[{'id': 15278, 'year': 2012, 'month': 8, 'amou...",1,Germany,Münster,transportation,[fund],[],None
33,223114,Quaturo,BlueGlass | SEO Content Marketing Agency in ...,BlueGlass | SEO & Content Marketing Agency in ...,1,1,1,1,0.00,acquired,[Subsidiary],0,NaN,[],"[{'id': 7794, 'name': 'BlueGlass Interactive',...",1,"[{'id': 39622, 'name': 'Kevin Gibbons', 'path'...",1,"[{'id': 223356, 'year': 2012, 'month': 11, 'am.

In [132]:
df_missing_univ["linkedin_url_founder"].count()

12666